In [128]:
import pandas as pd
import numpy as np
import time
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

In [127]:
def get_mols(smiles):
        try:
            mol = Chem.MolFromSmiles(smiles)
#             print(".",end="")
        except Exception as e:
            print(e,'\t',smiles,'\n')
            mol = None
        return mol

In [126]:
nms=[x[0] for x in Descriptors._descList]
print(len(nms))
mol = Chem.MolFromSmiles('CN(C)CCCN1c2ccccc2CCc2ccc(Cl)cc21') #example
calc = MoleculeDescriptors.MolecularDescriptorCalculator(nms)
type(calc.CalcDescriptors(mol))

208


tuple

In [108]:
def get_descriptors(m):
        try:
            calc = MoleculeDescriptors.MolecularDescriptorCalculator(nms)
            descrs = calc.CalcDescriptors(m)
#             print(".",end="")
        except Exception as e:
            print(e,'\t',m,'\n')
            descrs = None
        return descrs

In [109]:
def process(df):
    begin = time.time()
    smiles_list = pd.Series(list(df['Canonical_SMILES']))
    descrs_list = list()
    df['Mol_Objects'] = smiles_list.map(get_mols)
    mol_list = pd.Series(list(df['Mol_Objects']))
    for mol in mol_list:
        descrs_list.append(get_descriptors(mol))
    df_temp = pd.DataFrame([x for x in descrs_list])
    df = pd.concat([df,df_temp],axis=1)
    end = time.time()
    elapsed = end - begin
    print(elapsed)
    return df

# DILI

In [142]:
df = pd.read_csv('./DILI/final/DILI_Final.csv')
df

,ID,Canonical_SMILES,DILI
0,ref1_00001,CNCC[C@@H](Oc1ccccc1C)c1ccccc1,1
1,ref1_00002,CN1CCC[C@@H]1CCO[C@](C)(c1ccccc1)c1ccc(Cl)cc1,0
2,ref1_00003,CN(C)CCCN1c2ccccc2CCc2ccc(Cl)cc21,1
3,ref1_00004,CN1CCN(C2=Nc3cc(Cl)ccc3Nc3ccccc32)CC1,1
4,ref1_00005,NC(=O)C(c1ccccc1)(c1ccccc1)[C@@H]1CCN(CCc2ccc3...,0
...,...,...,...
2256,ref6_01298,CN1C[C@H](C(=O)N[C@]2(C)O[C@@]3(O)[C@@H]4CCCN4...,0
2257,ref6_01300,CNC[C@H](O)c1ccc(O)c(O)c1,0
2258,ref6_01301,CCCCCCCC/C=C/CCCCCCCC(=O)O,0
2259,ref6_01304,NC(N)=NCCN1CCCCCCC1,0


In [143]:
df = process(df)
df_a=df[['ID','Canonical_SMILES','DILI']]
df_b=df.loc[:,0:207]
recolumn = []
for i in range(208):
    recolumn.append(f'{nms[i]}')
df_b.columns = recolumn
df = pd.concat([df_a,df_b],axis=1)
df

58.322752475738525


,ID,Canonical_SMILES,DILI,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,ref1_00001,CNCC[C@@H](Oc1ccccc1C)c1ccccc1,1,6.193380,0.092361,6.193380,0.092361,0.848995,255.361,234.193,...,0,0,0,0,0,0,0,0,0,0
1,ref1_00002,CN1CCC[C@@H]1CCO[C@](C)(c1ccccc1)c1ccc(Cl)cc1,0,6.482112,-0.455333,6.482112,0.455333,0.722715,343.898,317.690,...,0,0,0,0,0,0,0,0,0,0
2,ref1_00003,CN(C)CCCN1c2ccccc2CCc2ccc(Cl)cc21,1,6.271931,0.815501,6.271931,0.815501,0.817874,314.860,291.676,...,0,0,0,0,0,0,0,0,0,0
3,ref1_00004,CN1CCN(C2=Nc3cc(Cl)ccc3Nc3ccccc32)CC1,1,6.176540,0.706466,6.176540,0.706466,0.800122,326.831,307.679,...,0,0,0,0,0,0,0,0,0,0
4,ref1_00005,NC(=O)C(c1ccccc1)(c1ccccc1)[C@@H]1CCN(CCc2ccc3...,0,13.195604,-0.817283,13.195604,0.137241,0.622271,426.560,396.320,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256,ref6_01298,CN1C[C@H](C(=O)N[C@]2(C)O[C@@]3(O)[C@@H]4CCCN4...,0,14.188117,-2.026666,14.188117,0.171241,0.433136,583.689,546.393,...,0,0,0,0,0,0,0,0,0,0
2257,ref6_01300,CNC[C@H](O)c1ccc(O)c(O)c1,0,9.480602,-0.669907,9.480602,0.180178,0.510157,183.207,170.103,...,0,0,0,0,0,0,0,0,0,0
2258,ref6_01301,CCCCCCCC/C=C/CCCCCCCC(=O)O,0,10.324075,-0.664234,10.324075,0.331507,0.290517,282.468,248.196,...,0,0,0,0,0,0,0,0,10,0
2259,ref6_01304,NC(N)=NCCN1CCCCCCC1,0,5.273144,0.205596,5.273144,0.205596,0.513143,198.314,176.138,...,0,0,0,0,0,0,0,0,0,0


In [144]:
df.to_csv('./DILI/final/DILI_DESCRS.csv')

In [145]:
mfps = pd.read_csv('./DILI/final/DILI_MFPS.csv')
descrs = pd.read_csv('./DILI/final/DILI_DESCRS.csv')
df_final = pd.merge(mfps,descrs)
df_final = df_final.loc[:,"ID":'fr_urea']
df_final

,ID,Canonical_SMILES,DILI,MFPS_1,MFPS_2,MFPS_3,MFPS_4,MFPS_5,MFPS_6,MFPS_7,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,ref1_00001,CNCC[C@@H](Oc1ccccc1C)c1ccccc1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ref1_00002,CN1CCC[C@@H]1CCO[C@](C)(c1ccccc1)c1ccc(Cl)cc1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ref1_00003,CN(C)CCCN1c2ccccc2CCc2ccc(Cl)cc21,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ref1_00004,CN1CCN(C2=Nc3cc(Cl)ccc3Nc3ccccc32)CC1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ref1_00005,NC(=O)C(c1ccccc1)(c1ccccc1)[C@@H]1CCN(CCc2ccc3...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256,ref6_01298,CN1C[C@H](C(=O)N[C@]2(C)O[C@@]3(O)[C@@H]4CCCN4...,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2257,ref6_01300,CNC[C@H](O)c1ccc(O)c(O)c1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2258,ref6_01301,CCCCCCCC/C=C/CCCCCCCC(=O)O,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,10,0
2259,ref6_01304,NC(N)=NCCN1CCCCCCC1,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [146]:
df_final.to_csv('./DILI/final/DILI_Features.csv')

## Model-Free

In [147]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
df = pd.read_csv('./DILI/final/DILI_Features_cleaned.csv', index_col=0)
df_split = df.iloc[:,1027:]
df_split

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,6.193380,0.092361,6.193380,0.092361,0.848995,255.361,234.193,255.162314,100,0,...,0,0,0,0,0,0,0,0,0,0
1,6.482112,-0.455333,6.482112,0.455333,0.722715,343.898,317.690,343.170292,128,0,...,0,0,0,0,0,0,0,0,0,0
2,6.271931,0.815501,6.271931,0.815501,0.817874,314.860,291.676,314.154976,116,0,...,0,0,0,0,0,0,0,0,0,0
3,6.176540,0.706466,6.176540,0.706466,0.800122,326.831,307.679,326.129824,118,0,...,0,0,0,0,0,0,0,0,0,0
4,13.195604,-0.817283,13.195604,0.137241,0.622271,426.560,396.320,426.230728,164,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256,14.188117,-2.026666,14.188117,0.171241,0.433136,583.689,546.393,583.279469,224,0,...,0,0,0,0,0,0,0,0,0,0
2257,9.480602,-0.669907,9.480602,0.180178,0.510157,183.207,170.103,183.089543,72,0,...,0,0,0,0,0,0,0,0,0,0
2258,10.324075,-0.664234,10.324075,0.331507,0.290517,282.468,248.196,282.255880,118,0,...,0,0,0,0,0,0,0,0,10,0
2259,5.273144,0.205596,5.273144,0.205596,0.513143,198.314,176.138,198.184447,82,0,...,0,0,0,0,0,0,0,0,0,0


In [148]:
scaler = MinMaxScaler()
scaler.fit(df_split)
scaled =  scaler.transform(df_split) 

In [149]:
df_scaled = pd.DataFrame(scaled)
df_scaled

,0,1,2,3,4,5,6,7,8,9,...,198,199,200,201,202,203,204,205,206,207
0,0.355380,0.839519,0.355380,0.014041,0.907059,0.033733,0.033098,0.033729,0.034226,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.371948,0.786495,0.371948,0.069219,0.770975,0.046317,0.045763,0.046246,0.044643,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.359887,0.909528,0.359887,0.123972,0.873523,0.042190,0.041817,0.042120,0.040179,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.354414,0.898972,0.354414,0.107396,0.854392,0.043891,0.044245,0.043823,0.040923,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.757172,0.751453,0.757172,0.020863,0.662734,0.058065,0.057691,0.058060,0.058036,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2221,0.814123,0.634369,0.814123,0.026032,0.458915,0.080398,0.080455,0.080396,0.080357,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2222,0.544003,0.765721,0.544003,0.027390,0.541915,0.023478,0.023376,0.023479,0.023810,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2223,0.592402,0.766270,0.592402,0.050395,0.305223,0.037586,0.035222,0.037583,0.040923,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.384615,0.0
2224,0.302576,0.850481,0.302576,0.031255,0.545133,0.025625,0.024291,0.025626,0.027530,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [150]:
describe = df_scaled.describe().T
greater_005 = describe['std'] > 0.05

In [151]:
describe = describe[greater_005]
describe

,count,mean,std,min,25%,50%,75%,max
0,2226.0,0.641750,0.163958,0.0,0.592254,0.685941,0.739124,1.0
1,2226.0,0.720684,0.133735,0.0,0.680727,0.744346,0.804935,1.0
2,2226.0,0.641750,0.163958,0.0,0.592254,0.685941,0.739124,1.0
4,2226.0,0.559851,0.250249,0.0,0.380334,0.587343,0.766220,1.0
5,2226.0,0.054520,0.056821,0.0,0.033294,0.045402,0.059440,1.0
...,...,...,...,...,...,...,...,...
202,2226.0,0.014376,0.119060,0.0,0.000000,0.000000,0.000000,1.0
203,2226.0,0.015274,0.093570,0.0,0.000000,0.000000,0.000000,1.0
205,2226.0,0.012354,0.081861,0.0,0.000000,0.000000,0.000000,1.0
206,2226.0,0.013252,0.059005,0.0,0.000000,0.000000,0.000000,1.0


In [152]:
describe.to_csv('./DILI/temp/Describe.csv')

In [153]:
filter = describe['std'] > 0.05
filter.index

Int64Index([  0,   1,   2,   4,   5,   6,   7,   8,  10,  11,
            ...
            197, 198, 199, 200, 201, 202, 203, 205, 206, 207],
           dtype='int64', length=181)

In [154]:
df_filtered = df_split.iloc[:,filter.index]
df_filtered

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,MaxPartialCharge,MinPartialCharge,...,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea
0,6.193380,0.092361,6.193380,0.848995,255.361,234.193,255.162314,100,0.124936,-0.485416,...,0,0,0,0,0,0,0,0,0,0
1,6.482112,-0.455333,6.482112,0.722715,343.898,317.690,343.170292,128,0.115262,-0.365775,...,0,0,0,0,0,0,0,0,0,0
2,6.271931,0.815501,6.271931,0.817874,314.860,291.676,314.154976,116,0.045791,-0.340948,...,0,0,0,0,0,0,0,0,0,0
3,6.176540,0.706466,6.176540,0.800122,326.831,307.679,326.129824,118,0.138322,-0.353471,...,0,0,0,0,0,0,0,0,0,0
4,13.195604,-0.817283,13.195604,0.622271,426.560,396.320,426.230728,164,0.232388,-0.492959,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256,14.188117,-2.026666,14.188117,0.433136,583.689,546.393,583.279469,224,0.280215,-0.360880,...,0,0,0,0,0,0,0,0,0,0
2257,9.480602,-0.669907,9.480602,0.510157,183.207,170.103,183.089543,72,0.157261,-0.504260,...,0,0,0,0,0,0,0,0,0,0
2258,10.324075,-0.664234,10.324075,0.290517,282.468,248.196,282.255880,118,0.302840,-0.481230,...,0,0,0,0,0,0,0,0,10,0
2259,5.273144,0.205596,5.273144,0.513143,198.314,176.138,198.184447,82,0.185451,-0.370152,...,0,0,0,0,0,0,0,0,0,0


In [155]:
df_label = df.iloc[:, 0:1027]
# df_label
df_free = pd.concat([df_label, df_filtered], axis=1)
df_free

,ID,Canonical_SMILES,DILI,MFPS_1,MFPS_2,MFPS_3,MFPS_4,MFPS_5,MFPS_6,MFPS_7,...,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea
0,ref1_00001,CNCC[C@@H](Oc1ccccc1C)c1ccccc1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ref1_00002,CN1CCC[C@@H]1CCO[C@](C)(c1ccccc1)c1ccc(Cl)cc1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ref1_00003,CN(C)CCCN1c2ccccc2CCc2ccc(Cl)cc21,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ref1_00004,CN1CCN(C2=Nc3cc(Cl)ccc3Nc3ccccc32)CC1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ref1_00005,NC(=O)C(c1ccccc1)(c1ccccc1)[C@@H]1CCN(CCc2ccc3...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256,ref6_01298,CN1C[C@H](C(=O)N[C@]2(C)O[C@@]3(O)[C@@H]4CCCN4...,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2257,ref6_01300,CNC[C@H](O)c1ccc(O)c(O)c1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2258,ref6_01301,CCCCCCCC/C=C/CCCCCCCC(=O)O,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,10,0
2259,ref6_01304,NC(N)=NCCN1CCCCCCC1,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [156]:
df_free.to_csv('./DILI/final/DILI_Model_Free.csv')

# Acute Oral Toxicity

In [157]:
df = pd.read_csv('./Acute Oral Toxicity/final/AOT_LD50_Final.csv')
df = process(df)
df_a=df[['ID','Canonical_SMILES','LD50_minLog']]
df_b=df.loc[:,0:207]
recolumn = []
for i in range(208):
    recolumn.append(f'{nms[i]}')
df_b.columns = recolumn
df = pd.concat([df_a,df_b],axis=1)
df

81.42358827590942


,ID,Canonical_SMILES,LD50_minLog,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,ref1_00001,O=[N+]([O-])c1ccc(Cl)cc1,-0.337242,10.074698,-0.461975,10.074698,0.059630,0.463602,157.556,153.524,...,0,0,0,0,0,0,0,0,0,0
1,ref1_00002,Nc1ccc([N+](=O)[O-])cc1,-0.124939,10.076965,-0.458889,10.076965,0.064074,0.359544,138.126,132.078,...,0,0,0,0,0,0,0,0,0,0
2,ref1_00003,O=[N+]([O-])c1ccc(O)cc1,-0.632201,10.036149,-0.514444,10.036149,0.015926,0.470728,139.110,134.070,...,0,0,0,0,0,0,0,0,0,0
3,ref1_00004,O=[N+]([O-])c1ccc(CCl)cc1,0.257439,10.173125,-0.432528,10.173125,0.096173,0.389482,171.583,165.535,...,0,0,0,0,0,0,0,0,0,0
4,ref1_00006,COc1ccc([N+](=O)[O-])cc1,0.361728,10.165294,-0.445000,10.165294,0.074815,0.478623,153.137,146.081,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8314,ref1_11885,c1ccccc1,0.521530,2.000000,2.000000,2.000000,2.000000,0.442628,78.114,72.066,...,0,0,0,0,0,0,0,0,0,0
8315,ref1_11886,COP(=O)(OC)OC=C(Cl)Cl,-1.522879,11.026605,-3.480301,11.026605,0.167292,0.540194,220.976,213.920,...,0,0,0,0,0,0,0,0,0,0
8316,ref1_11887,CCOP(=S)(OCC)Oc1cc(C)nc(C(C)C)n1,-1.036212,5.684815,-2.768105,5.684815,0.221566,0.717616,304.352,283.184,...,0,0,0,0,0,0,0,0,0,0
8317,ref1_11888,CC(CCl)OP(=O)(OC(C)CCl)OC(C)CCl,-0.008774,12.278519,-3.690648,12.278519,0.181768,0.473671,327.572,309.428,...,0,0,0,0,0,0,0,0,0,0


In [158]:
df.to_csv('./Acute Oral Toxicity/temp2/AOT_LD50_DESCRS.csv')

In [159]:
mfps = pd.read_csv('./Acute Oral Toxicity/final/AOT_LD50_MFPS.csv')
descrs = pd.read_csv('./Acute Oral Toxicity/temp2/AOT_LD50_DESCRS.csv')
df_final = pd.merge(mfps,descrs)
df_final = df_final.loc[:,"ID":'fr_urea']
df_final

FileNotFoundError: [Errno 2] No such file or directory: './Acute Oral Toxicity/final/AOT_LD50_MFPS.csv'

In [ ]:
df_final.to_csv('./DILI/final/DILI_Features.csv')